## Tuning Hyperparameters with Keras Tuner 

- Task: Find the best learning rate for the Anomaly Detection
- Dataset: Cropped and Resized Fundus Images
- Original Colorscheme

Ressources for use and problem solving
- Tensorflow: https://www.tensorflow.org/tutorials/keras/keras_tuner
- Keras: https://keras.io/api/keras_tuner/hyperparameters/
- Hyperband Tuner: https://arxiv.org/abs/1603.06560
- https://stackoverflow.com/questions/62258704/what-does-infotensorfloworacle-triggered-exit-mean-with-keras-tuner
- https://github.com/keras-team/keras-tuner/issues/104
- https://github.com/keras-team/keras-tuner/issues/223


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import tensorflow as tf

In [3]:
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import EarlyStopping
import keras_tuner as kt

In [4]:
# read in data
df = pd.read_csv('train_binary.csv')

In [5]:
# Assign str Datatype to target for Generator
df.normal = df.normal.astype('str')

In [6]:
# Check Data
df.normal.value_counts()

1    3150
0    2910
Name: normal, dtype: int64

In [7]:
# As data is ordered in dataframe after augmentation, shuffle it
from sklearn.utils import shuffle 
df = shuffle(df)

In [8]:
# Directory
indir = r"D:\data\Projects\notebooks\RetinaAI\Anomaly Detection\train"

In [9]:
# Normalize Data
datagen = ImageDataGenerator(rescale=1./255., validation_split = 0.25)

In [10]:
train_gen = datagen.flow_from_dataframe(dataframe = df, 
                                            directory = indir, 
                                            x_col = "filename", 
                                            y_col = 'normal',
                                            batch_size = 64, 
                                            seed = 2, 
                                            shuffle = True, 
                                            class_mode = "binary", 
                                            target_size = (300,300),
                                            subset='training')

Found 4545 validated image filenames belonging to 2 classes.


In [11]:
val_gen = datagen.flow_from_dataframe(dataframe = df, 
                                            directory = indir, 
                                            x_col = "filename", 
                                            y_col = 'normal',
                                            batch_size = 64, 
                                            seed = 2, 
                                            shuffle = True, 
                                            class_mode = "binary", 
                                            target_size = (300,300),
                                            subset='validation')

Found 1515 validated image filenames belonging to 2 classes.


In [12]:
# Model builder with selection of learning rates
def builder(hp):
    
    model = Sequential()

    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(300,300,3)))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))

    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))

    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))

    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(1, activation='sigmoid'))

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), 
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])
    
    return model

In [13]:
tuner = kt.Hyperband(builder,
                     objective = 'val_accuracy',
                     max_epochs=10,
                     directory = 'my_dir',
                     project_name = 'anom_tun')

In [14]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [15]:
step_size_train = train_gen.n//train_gen.batch_size
step_size_val = val_gen.n//val_gen.batch_size

The Hyperband tuning algorithm uses adaptive resource allocation and early-stopping to quickly converge on a high-performing model.

In [16]:
tuner.search(x=train_gen, 
             validation_data=val_gen, 
             steps_per_epoch=step_size_train, 
             validation_steps=step_size_val, 
             epochs=20, 
             callbacks=[stop_early])

Trial 3 Complete [01h 49m 47s]
val_accuracy: 0.9728260636329651

Best val_accuracy So Far: 0.9728260636329651
Total elapsed time: 05h 28m 02s
INFO:tensorflow:Oracle triggered exit
